In [149]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import binascii
import math
import numpy as np
import importlib
from libraries import crypto
from libraries import numeral

importlib.reload(crypto)
importlib.reload(numeral)

<module 'libraries.numeral' from 'C:\\Users\\Oguz\\Desktop\\Cryptopals Challange\\libraries\\numeral.py'>

<p style="color:royalblue; font-size:18px;">
    <b>
        Loading the Challange 6 Data
    </b>
</p>

In [150]:
challange_data_addr = "C:\\Users\\Oguz\\Desktop\\Cryptopals Challange\\challanges\\challange_6.txt"
data = np.loadtxt(challange_data_addr, dtype='str')
print("Data length is", len(data))
first_string = data[0]
first_string_len = len(first_string)
print("String: ", first_string)
print("Length: ", first_string_len)
first_string[-1]

Data length is 64
String:  HUIfTQsPAh9PE048GmllH0kcDk4TAQsHThsBFkU2AB4BSWQgVB0dQzNTTmVS
Length:  60


'S'

<p style="color:royalblue; font-size:18px;">
    <b>
        Finding the Hamming Distance
    </b>
</p>

In [151]:
test_plaintext = "this is a test"
test_key = "wokka wokka!!!"


distance_deneme = crypto.Crypto(plaintext=test_plaintext)

distance_deneme.findHammingDistance(test_key)

0111011101101111011010110110101101100001001000000111011101101111011010110110101101100001001000010010000100100001


37

<p style="color:royalblue; font-size:18px;">
    <b>
        Finding the Keysize
    </b>
</p>

For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.

The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.

In [152]:
distance_deneme.Base2()
test_bytes = distance_deneme.value
print(test_bytes)

keysize = 3


crypto.Crypto.findKeysizeScore(keysize, test_bytes)

0111010001101000011010010111001100100000011010010111001100100000011000010010000001110100011001010111001101110100


1.6666666666666667

<p style="color:royalblue; font-size:18px;">
    <b>
        Finding the Keysize for the Challange 6 Text
    </b>
</p>

<p style="color:block; font-size:14px;">
    <b>
      <p> For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
      </p>
      <p>The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the 
        smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
        </p>
    </b>
</p>

In [153]:
importlib.reload(crypto)


data_str = "" 

for elem in data:
    data_str += elem
data_str = data_str[:-1]
    
challange_6_numeral = numeral.Numeral(value=data_str, base = 64)
challange_6_numeral.Base16()

challange_6 = crypto.Crypto(ciphertext = challange_6_numeral.value)

challange_6_scores = challange_6.findKeysize()
print(challange_6_scores)

N = 3
min_score_idx = np.array(challange_6_scores).argsort()[:N][::1]

min_score_idx

len(challange_6.value)
print(len(data_str))

[6.         2.5        2.         3.5        1.6        4.16666667
 3.42857143 2.625      3.55555556 3.3        3.09090909 3.5
 2.76923077 3.28571429 2.93333333 3.25       2.88235294 2.94444444
 3.36842105 3.         2.71428571 3.40909091 3.39130435 3.33333333
 3.36       3.61538462 3.40740741 3.53571429 3.27586207 3.5
 3.22580645 3.5        3.39393939 3.44117647 3.42857143 3.52777778
 3.10810811 3.07894737 3.25641026 3.25      ]
3835


<p style="color:royalblue; font-size:18px;">
    <b>
        Finding the Key
    </b>
</p>

<p style="color:block; font-size:14px;">
    <b>
      <p> Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.</p>
      <p>Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on. Solve each block as if it was single-character XOR. You already have code to do this.</p>
    </b>
</p>

In [154]:
keysize_cand = min_score_idx[0] + 1
blocksize = keysize_cand * 8
bin_data = challange_6.value

# Add 0's to the and of the data so that it is evenly divisible to KEYSIZE
bin_data_size = len(bin_data)
remainder = bin_data_size % (blocksize*5)
for i in range(abs(blocksize - remainder)):
    bin_data += "0"
bin_data_size = len(bin_data)

# Create number of numerals to hold each block
numeral_objs = [numeral.Numeral(value="", base=2) for i in range(keysize_cand)]

# Break the ciphertext into blocks of KEYSIZE length.
block_no = int(bin_data_size / blocksize)

# In the first loop divide the data into (Keysize*8) bits
for i in range(block_no):
    dummy_block = bin_data[(i*blocksize):(i+1)*blocksize]
    #In the second loop divide each block, and create a block of first, second, etc bytes
    for j in range(keysize_cand):
        dummy_chunk = dummy_block[(j*8):(j+1)*8]
        numeral_objs[j].value = numeral_objs[j].value + dummy_chunk


# Convert the numeral_objs to Base16
for i in range(keysize_cand):
    numeral_objs[i].Base16()

In [162]:
importlib.reload(crypto)


# Solve each block / Create KEYSIZE crypto instances for each block with the numeral_objs
crypto_objs = [crypto.Crypto(ciphertext=numeral_objs[i].value) for i in range(keysize_cand)]


len(crypto_objs[0].value)

for i in range(keysize_cand):
    crypto_objs[i].decipherSingleByteXOR(print_sln=True, N=1)
    print("***")

Key:  E  / Solution:  XJZ^EeCHI[T_XL&IELM'_1_>_DwCNnBBM\S,GBDG^ryJEY!IKxHKNBNEZÈKÂCoBtOXVKNAY_MV)JiB\^oGSMR EpDHSEEuo'%'TËCNRX UVk!0F×EOLLuÌCGBeXOCXdyTKIMtC^ARE_Cc^EMDR{xËsPBc@rCCVsXGÜmV\^&%Hd_R^s^BOXT_XOEAdbJ~DX ¨Yáeead]YÈH-Uµ}Q}@ÁXEY5qmQI¸e9A`Ea%EEDaÙa(üí-hE,]y¤7ap]±`Ôu]E\-uÅEYÝ50deA}\=M!}w-AAðEah-=IEÙmPµq4E,|tHEELE aµÍE¥0-YDUUõa4U5,9)øqXÿ¡UQe-Y{LûyU±mia5@EqIAdMÉPÍu­h!)EQAéL7IíqlQMA<|}Tii1@hlQôxelQ,]Xu%TD}Ue4¥Õ½Es]es±aQe=5xIq5=íQaiéuhAu-AM`tea)]<p
***
Key:  E  / Solution:  G8DD[EÖMEE_e|_rH^AL:EHVBb<DIM cBCGUG@CHYOEIk'orBRF}BDÌ+_IB"OÃgERDVSB{GZ\DSLo_aLC,=bH~J^ÑcO]y@EMVC_bXEM:ÚhuYÁ\]YEw&_[RHN[TOV'JsNyXKE=IEITX^BF_NkEtY\T^TrLBVNS_eE[sJCXÊNIü_YDQq¨ $T)}mYuy8\EQY1áyYE]õeY]A-)pu1IXE4wTÉDñ-}1Ypô]{,ÈqadQ»ð) Q|Xl5@mé°\%a)¼{iM3u}Ex|ME¡5AAÍYµeEX 7,AThp]­E5 Y|exE]0EmA]Xa4%-mP`eEñDEÍÉA¬LÕéuQdaeuD,\­alIXD]50õ|hAL5pø 05]°,Lae1íd|EpÐ`|Es]4A±P8a%)dQpY|8l5Íalð,iA1ED÷yel,ME@u}<QÿE¹ÅeEÀ1yE¬e]5,|Ü5M@A\}É
***
Key:  e  / Solution:  zzyns$x+"(w,AN<Rr1v7 #iíe{t<à6|cl$V`8B

In [176]:
challange_6_key = "EEeeE"

ciphertext_6 = data_str


challange_6_v2 = crypto.Crypto(ciphertext=ciphertext_6)
challange_6_v2.base = 64
challange_6_v2.Base16()


# # a = challange_5.encodeRepeatingByteXOR(key)
# # print(a == cipher_5)

b = challange_6_v2.decodeRepeatingByteXOR(challange_6_key)
print(b)



Xz(NJGzêV8 ZykDnb^Ds rE[$,!ex8+ CE"1YH ("AIÖwYI[M,LT
A)d_Ne@X<tALECB&ERCCI_r$X1UEevo@7-M  ZSL| c@M_#b/'rix
Ev_
Me*UMnX\geKSe7u, zìGGedNB'uDcupGBgzÊ^ÜU,rodLCblGu6MyUc`J
u*9E:"^YQ
!Gu6_
@st1IC(*GK&	x%JNHm+RH1xÊKYà;mN|dWBO06<
ÇcDNxIEEro@ZI^'Èkd$gK,Q'Sc
ÂoM
~drabk	pyKCBNOoR,|CD BF`6Bt IBOx,}*ecXB,vEVDv KÌraN+èxCA_*Ü2e^XY11P_I+ì
iEh%BBR^l\Dn$B^V2[_oSiYGBb-N
neES{n MGh~mRU, Z,bLE\t 
t-NSz/gEeiËE_n7u^om.'a+o
%cSN'LmeGT
5 iË U7XCC+1LN,+aER _'X jM =Ib>H1TE>dS ~nrUJ#1]V>Ek^OGÑyoE!cx1S0OqnF]1bRy6X×@nvEEEeeIOMb aLVA,_LCebu_OtXÌb2&PC #~DGiE BX,jJe
1~O*KOu nCY$vEXÁ3d\ +k]dk u6RyYzm_EexXw00w&[wT_c$GK
Aa_ RNjnGE
Ce*NcT
y+f	X&eE{KaeO :5xE|yXË=+ê'sIgkÉPE OBIde6cfc@TdnrXiÇnC^"6PC 1ySBoHVFk+îs_ced {@	 t] ysEXtXn@GNeNÜkr,!mt En,YVtr;\Yer^^[&\ix%Tf H^,bdT7fC_xlr*bRjâ	^Lf0Y
aY X$y,é9t|D\$eÜXila5KDQMuH)@-ÍdAí1]\45|}q<a<ùU pÉ
